In [47]:
from helpers import load_CSD_data, make_element_wise_environments
import numpy as np
from rascal.representations import SphericalInvariants
from rascal.models.kernels import Kernel
from sklearn.kernel_ridge import KernelRidge
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species

PATH_TRAIN = "CSD-2k_relaxed_shifts.txt"
PATH_TEST = "CSD-500.txt"
structures_test, shifts_test = load_CSD_data(PATH_TEST,random_subsample=50)
structures_train, shifts_train = load_CSD_data(PATH_TRAIN,random_subsample=200)

In [48]:
hypers = {"soap_type": "PowerSpectrum",
          "interaction_cutoff": 3,
          "radial_basis": "GTO",
          "max_radial": 9,
          "max_angular": 9,
          "gaussian_sigma_constant": 0.3,
          "gaussian_sigma_type":"Constant",
          "cutoff_function_type":"ShiftedCosine",
          "cutoff_smooth_width": 0.5,
          "normalize": True,
          "compute_gradients":False,
          "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4),
          #"optimization": dict(Spline=dict(accuracy=1.0e-05))
          "expansion_by_species_method":'user defined',
          "global_species":[1,6,7,8]
         }


In [49]:
! export OMP_NUM_THREADS=1
! export MKL_NUM_THREADS=1

In [3]:
calculator = SphericalInvariants(**hypers)
atoms_list_train = calculator.transform(structures_train)
#X_repr_new = atoms_list_new.get_features(calculator)

In [35]:
for structure in structures_train: mask_center_atoms_by_species(structure,species_select=[1])
calculator = SphericalInvariants(**hypers)
atoms_list_train = calculator.transform(structures_train)
kernel = Kernel(calculator,target_type="Atom",zeta=2)
#kernel_computed = kernel(atoms_list_train)

In [21]:
kernel_computed.shape

(3694, 3694)

In [31]:
#masking the species
#for structure in structures_train: mask_center_atoms_by_species(structure,species_select=[1])
H_train_shifts = np.array([shift for structure in structures_train for shift in structure.arrays["CS"][structure.arrays["center_atoms_mask"]]])    

KeyError: 'center_atoms_mask'

In [28]:


kernel = Kernel(calculator,target_type="Atom",zeta=2)
kernel_computed = kernel(atoms_list_train)

#clf = KernelRidge(alpha=1e-02,kernel="precomputed")

#clf.fit(kernel_computed, H_train_shifts)

In [37]:
X_train = calculator.transform(structures_train).get_features(calculator)
X_test = calculator.transform(structures_test).get_features(calculator)

In [13]:
import sklearn

In [24]:
Kernel_sklearn = sklearn.metrics.pairwise.polynomial_kernel(X_train, degree=2, gamma=1., coef0=0)

In [14]:
Kernel_sklearn.shape

(3928, 3928)

In [25]:
np.allclose(kernel_computed,Kernel_sklearn) #returns True

True

In [9]:
import cProfile

In [45]:
cProfile.run("kernel(atoms_list_train)")

         5 function calls in 30.057 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.065    0.065   30.055   30.055 <string>:1(<module>)
        1   29.990   29.990   29.990   29.990 kernels.py:114(__call__)
        1    0.001    0.001   30.057   30.057 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [46]:
cProfile.run("X_train = calculator.transform(structures_train).get_features(calculator); sklearn.metrics.pairwise.polynomial_kernel(X_train, degree=2, gamma=1., coef0=0)")

         9106 function calls in 8.811 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      200    0.000    0.000    0.006    0.000 <__array_function__ internals>:2(all)
        1    0.000    0.000    0.052    0.052 <__array_function__ internals>:2(sum)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:416(parent)
        1    0.226    0.226    8.810    8.810 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
        1    0.000    0.000    0.000    0.000 _asarray.py:110(asanyarray)
      401    0.000    0.000    0.003    0.000 _asarray.py:23(asarray)
        1    0.000    0.000    0.000    0.000 _config.py:16(_get_threadlocal_config)
        1    0.000    0.000    0.000    0.000 _config.py:24(get_config)
        2    0.000    0.000    0.002    0.001 abc.py:137(__

In [65]:
cProfile.run("clf.fit(kernel_computed, H_train_shifts)")

         358 function calls in 1.674 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(atleast_1d)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(atleast_2d)
        3    0.000    0.000    0.236    0.079 <__array_function__ internals>:2(sum)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(unique)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:416(parent)
        1    0.012    0.012    1.674    1.674 <string>:1(<module>)
        9    0.000    0.000    0.000    0.000 _asarray.py:110(asanyarray)
        5    0.000    0.000    0.000    0.000 _asarray.py:23(asarray)
        3    0.000    0.000    0.000    0.000 _config.py:16(_get_threadlocal_config)
        3    0.0

In [62]:
kernel_computed.shape

(3684, 3684)

In [16]:
for structure in structures_test: mask_center_atoms_by_species(structure,species_select=[1])
H_test_shifts = np.array([shift[0] for structure in structures_test for shift in structure.arrays["CS"][structure.arrays["center_atoms_mask"]]]) 

In [17]:
atom_list_test = calculator.transform(structures_test)

In [88]:
kernel_test = kernel(atoms_list_train,Y=atom_list_test)

In [89]:
y_predictions = clf.predict(kernel_test.T)

In [55]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [91]:
np.sqrt(2*mean_squared_error(H_test_shifts,y_predictions))

1.4245174413573556

In [99]:
clf = KernelRidge(alpha=1.,kernel="poly",degree=3,gamma=1.,coef0=1)

clf.fit(X_train,H_train_shifts)

KernelRidge(alpha=1.0, gamma=1.0, kernel='poly')

In [60]:
cProfile.run("clf.fit(X_train,H_train_shifts)")

         358 function calls in 3.393 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(atleast_1d)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(atleast_2d)
        3    0.001    0.000    1.226    0.409 <__array_function__ internals>:2(sum)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(unique)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        4    0.001    0.000    0.001    0.000 <frozen importlib._bootstrap>:416(parent)
        1    0.012    0.012    3.392    3.392 <string>:1(<module>)
        9    0.000    0.000    0.000    0.000 _asarray.py:110(asanyarray)
        5    0.001    0.000    0.001    0.000 _asarray.py:23(asarray)
        3    0.000    0.000    0.000    0.000 _config.py:16(_get_threadlocal_config)
        3    0.0

In [39]:
cProfile.run("X_test = calculator.transform(structures_test).get_features(calculator)")

         2683 function calls (2583 primitive calls) in 3.136 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       50    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(all)
       50    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(copyto)
       50    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(empty_like)
       50    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(ones_like)
        1    0.213    0.213    3.136    3.136 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
      100    0.000    0.000    0.001    0.000 _asarray.py:23(asarray)
        1    0.000    0.000    0.000    0.000 abc.py:137(__instancecheck__)
       50    0.000    0.000    0.001    0.000 arraywrapper.py:66(attr)
       50    0.000    0.000    0.000    0.000 atoms.py:1947(cell)
       50    0.000    0.000    0.001    0.000 atoms.py:39

In [94]:
np.sqrt(2*mean_squared_error(H_test_shifts,clf.predict(X_test)))

1.424517441357186

In [100]:
np.sqrt(2*mean_squared_error(H_test_shifts,clf.predict(X_test)))

1.3884203203710437

In [56]:
mean_absolute_error(H_test_shifts,clf.predict(X_test))

0.7229663159244093

In [57]:
mean_absolute_error(H_test_shifts,y_predictions)

0.7341750449242127